# Popularity - Popular products

A simple rank of most seen products and a simple rank of most sold products. Both results combined is the popularity rank.

### Initialize Django

In [ ]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "shuup_workbench.settings")
import django
django.setup()

### Remove old data

In [ ]:
from shuup.core.models import *

OrderLine.objects.all().delete()
Order.objects.all().delete()
PersonContact.objects.all().delete()
Product.objects.all().delete()

### Create fake data

In [ ]:
import random
import uuid
from django.db.transaction import atomic
from shuup.core.models import *
from shuup.testing import factories


shop = Shop.objects.first()
supplier = factories.get_default_supplier()


with atomic():
    customers = [
        factories.create_random_person()
        for index in range(20)
    ]
    products = [
        factories.create_product(
            sku=uuid.uuid4().hex,
            shop=shop,
            supplier=supplier,
            default_price=random.randrange(1, 100)
        )
        for index in range(100)
    ]
    orders = [
        factories.create_random_order(
            customer=random.choice(customers),
            products=products,
            completion_probability=1,
            shop=shop
        )
        for index in range(200)
    ]

### Import libraries

In [ ]:
import pandas as pd
import numpy as np
from django_pandas.io import read_frame

## 1. Get most sold products

In [ ]:
from shuup.core.models import OrderLine, OrderStatus

order_lines = OrderLine.objects.products().filter(
    order__status=OrderStatus.objects.get_default_complete(),
    order__shop=shop
)
items_df = read_frame(order_lines, fieldnames=["product_id", "quantity"])

# group by product ID and sum quantities
sold_items = items_df.groupby(["product_id"]).sum()

# get the max value of quantity
max_value = sold_items.agg({"quantity": "max"})["quantity"]

# normalize the valus
sold_items["rank"] = sold_items["quantity"] / max_value

# remove quantity col
del(sold_items["quantity"])

# sort values by rank
sold_items.sort_values("rank", ascending=False)